<a href="https://colab.research.google.com/github/MEDINI7/ds_kopparapu_medini/blob/main/notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trader Behavior vs Market Sentiment Analysis



In [1]:
!git clone https://github.com/MEDINI7/ds_kopparapu_medini.git
%cd ds_kopparapu_medini

Cloning into 'ds_kopparapu_medini'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 65 (delta 20), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 737.59 KiB | 6.47 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Filtering content: 100% (4/4), 91.56 MiB | 21.93 MiB/s, done.
/content/ds_kopparapu_medini


Imports and setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.style.use("default")
pd.set_option("display.max_columns", None)

os.makedirs("csv_files", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

Loading the Datasets.

*   Here we load the trade data and the sentiment data and quickly check that both files have been read correctly.



In [3]:
trades = pd.read_csv("csv_files/historical_data.csv")
sentiment = pd.read_csv("csv_files/fear_greed_index.csv")

print("Trades shape:", trades.shape)
print("Sentiment shape:", sentiment.shape)

trades.head(), sentiment.head()

Trades shape: (211224, 16)
Sentiment shape: (2644, 4)


(                                      Account  Coin  Execution Price  \
 0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
 1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
 2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
 3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
 4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   
 
    Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
 0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
 1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
 2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
 3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
 4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   
 
    Closed PnL                                   Transaction Hash     Order ID  \
 0      

Previewing the data

*   This step is just to see what columns are present in both datasets.



In [4]:
print("Trades columns:")
print(trades.columns)

print("\nSentiment columns:")
print(sentiment.columns)


Trades columns:
Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp'],
      dtype='object')

Sentiment columns:
Index(['timestamp', 'value', 'classification', 'date'], dtype='object')


Viewing of rows

In [5]:
print("Trades shape:", trades.shape)
print("Sentiment shape:", sentiment.shape)


Trades shape: (211224, 16)
Sentiment shape: (2644, 4)


Data cleaning and preprocessing

*   Here we fix the date formats, standardize the sentiment labels, keep only the needed columns, and remove trades with missing values so the data is ready for analysis.



In [6]:
# Converting trade timestamp to datetime
trades["Timestamp"] = pd.to_datetime(trades["Timestamp"], unit="ms", errors="coerce")

# Creating date column
trades["date"] = trades["Timestamp"].dt.date

# Converting sentiment date to datetime
sentiment["date"] = pd.to_datetime(sentiment["date"], errors="coerce").dt.date

# Normalizing sentiment labels
sentiment["classification"] = sentiment["classification"].str.lower()

# Keeping only required sentiment columns
sentiment_clean = sentiment[["date", "classification"]]

# Droping of the trades with missing PnL or timestamp
trades_clean = trades.dropna(subset=["Closed PnL", "date"])

print("Trades clean shape:", trades_clean.shape)
print("Sentiment clean shape:", sentiment_clean.shape)

trades_clean.head(), sentiment_clean.head()


Trades clean shape: (211224, 17)
Sentiment clean shape: (2644, 2)


(                                      Account  Coin  Execution Price  \
 0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
 1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
 2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
 3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
 4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   
 
    Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
 0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
 1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
 2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
 3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
 4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   
 
    Closed PnL                                   Transaction Hash     Order ID  \
 0      

Combining trade data with market sentiment

*   In this step, we join both datasets on the date column to link each trade with its market sentiment.



In [7]:
df = trades_clean.merge(sentiment_clean, on="date", how="inner")

print("Merged shape:", df.shape)

df.to_csv("csv_files/merged_data.csv", index=False)

df.head()

Merged shape: (184263, 18)


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp,date,classification
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,2024-10-27 03:33:20,2024-10-27,greed
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,2024-10-27 03:33:20,2024-10-27,greed
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,2024-10-27 03:33:20,2024-10-27,greed
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,2024-10-27 03:33:20,2024-10-27,greed
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,2024-10-27 03:33:20,2024-10-27,greed


Creating a Win/Loss Indicator

*   Here we create a simple flag to mark each trade as a win or a loss based on whether the PnL is positive, which will help us calculate win rates later.



In [8]:
# Create win/loss flag
df["is_win"] = df["Closed PnL"] > 0

# Basic sanity check
df["is_win"].value_counts(normalize=True)


,proportion
is_win,
False,0.57965
True,0.42035


Daily Performance Summary

*   In this step, we summarize the trades at a daily level to make the analysis easier.



In [11]:
account_stats = df.groupby("Account").agg(
    total_pnl=("Closed PnL", "sum"),
    win_rate=("is_win", "mean"),
    avg_leverage=("Start Position", "mean"),
    trade_count=("Closed PnL", "count"),
    avg_position_size=("Size USD", "mean")
)

account_stats["pnl_volatility"] = df.groupby("Account")["Closed PnL"].std()

account_stats.head()

,total_pnl,win_rate,avg_leverage,trade_count,avg_position_size,pnl_volatility
Account,,,,,,
0x083384f897ee0f19899168e3b1bec365f52a9012,1.600230e+06,0.359612,-1247.334684,3818,16159.576734,4692.452329
0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,3.770617e+04,0.443031,19907.873157,3853,1759.743439,180.948061
0x271b280974205ca63b716753467d5a371de622ab,3.176309e+04,0.430192,3.314578,1146,10908.183944,119.639267
0x28736f43f1e871e6aa8b1148d38d4994275d72c4,1.323155e+05,0.438112,17874.870562,13266,509.016502,113.460234
0x2c229d22b100a7beb69122eed721cee9b24011dd,1.686280e+05,0.519790,473140.294007,3234,3142.909573,256.019993


Creating Trader-Level Performance Metrics

* Here we aggregate trade-level data at the account level to evaluate overall trader behavior. By calculating metrics such as total PnL, win rate, trade count, average position size, and PnL volatility, we shift the focus from individual trades to overall trader performance, which helps in identifying behavioral patterns and risk profiles for further analysis.

In [12]:
# Creating activity segment (top 25% as Frequent)
activity_threshold = account_stats["trade_count"].quantile(0.75)

account_stats["activity_segment"] = account_stats["trade_count"].apply(
    lambda x: "Frequent" if x >= activity_threshold else "Infrequent"
)

account_stats["activity_segment"].value_counts()

,count
activity_segment,
Infrequent,24
Frequent,8


In [13]:
df = df.merge(
    account_stats[["activity_segment"]],
    left_on="Account",
    right_index=True
)

Creating Activity-Based Segments

* Traders are segmented based on trading frequency. Accounts in the top 25% by trade count are classified as Frequent traders, while the rest are labeled as Infrequent traders. This helps analyze whether higher activity levels correspond to different performance patterns under Fear and Greed regimes.

In [14]:
segment_performance = df.groupby(
    ["classification", "activity_segment"]
).agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("is_win", "mean"),
    trades=("Closed PnL", "count")
)

segment_performance

avg_pnl  win_rate  trades
classification activity_segment                              
extreme greed  Frequent           31.134376  0.519699    6117
               Infrequent        -15.956790  0.275740     845
fear           Frequent           21.837182  0.419802   86405
               Infrequent        101.400655  0.406670   47466
greed          Frequent           82.425687  0.474055   29408
               Infrequent        111.268993  0.328586    6881
neutral        Frequent            9.793220  0.290529    5459
               Infrequent         62.592860  0.403686    1682

Strategy Recommendations

* Activity-Based Allocation: During extreme greed phases, capital should be allocated more heavily toward frequent traders, as they demonstrate higher win rates and positive average returns compared to infrequent traders.
* Cautious Execution in Neutral Markets: Frequent traders underperform during neutral sentiment periods, suggesting that reduced trade frequency may improve risk-adjusted performance in low-conviction environments.
* Selective Positioning During Fear: Infrequent traders appear to generate higher average PnL during fear regimes, indicating that fewer but higher-conviction trades may be more effective during high-uncertainty conditions.

In [15]:
behavior_table = df.groupby("classification").agg(
    avg_trade_size=("Size USD", "mean"),
    win_rate=("is_win", "mean"),
    trades=("Closed PnL", "count")
)
behavior_table

,avg_trade_size,win_rate,trades
classification,,,
extreme greed,5660.265764,0.490089,6962
fear,5259.977837,0.415146,133871
greed,3182.883845,0.446471,36289
neutral,3058.848110,0.317182,7141


Behavioral Shifts Across Sentiment Regimes

* Trade size tends to increase during extreme greed and fear regimes, suggesting higher conviction or risk-taking behavior during strong emotional market phases.
* Win rates are highest during extreme greed and lowest during neutral sentiment, indicating that strong directional sentiment may create clearer trading opportunities.
* Trade volume significantly increases during fear, reflecting heightened market activity and volatility during uncertain conditions.

This section highlights how trader behavior systematically shifts depending on the prevailing sentiment environment.

In [16]:
risk_table = df.groupby("classification").agg(
    avg_pnl=("Closed PnL", "mean"),
    pnl_volatility=("Closed PnL", "std"),
    win_rate=("is_win", "mean")
)

risk_table

,avg_pnl,pnl_volatility,win_rate
classification,,,
extreme greed,25.418772,306.166937,0.490089
fear,50.047622,909.121796,0.415146
greed,87.894859,1148.343968,0.446471
neutral,22.229713,633.704815,0.317182


Risk and Profitability Across Sentiment Regimes

* Average PnL is highest during greed regimes, suggesting stronger directional trends during optimistic market phases.
* Volatility (PnL dispersion) is significantly elevated during fear and greed periods, indicating increased uncertainty and larger outcome variability.
* Neutral sentiment shows the lowest win rate and moderate volatility, suggesting lower conviction and less predictable trading conditions.

This confirms that trader performance and risk exposure are highly regime-dependent

In [17]:
# Creating consistency segment (win rate based)
win_threshold_high = account_stats["win_rate"].quantile(0.75)
win_threshold_low = account_stats["win_rate"].quantile(0.25)

def consistency_segment(x):
    if x >= win_threshold_high:
        return "Consistent Winner"
    elif x <= win_threshold_low:
        return "Inconsistent Trader"
    else:
        return "Mid"

account_stats["consistency_segment"] = account_stats["win_rate"].apply(consistency_segment)

account_stats["consistency_segment"].value_counts()

,count
consistency_segment,
Mid,16
Consistent Winner,8
Inconsistent Trader,8


In [18]:
df = df.merge(
    account_stats[["consistency_segment"]],
    left_on="Account",
    right_index=True
)

In [19]:
consistency_performance = df.groupby(
    ["classification", "consistency_segment"]
).agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("is_win", "mean"),
    trades=("Closed PnL", "count")
)

consistency_performance

avg_pnl  win_rate  trades
classification consistency_segment                              
extreme greed  Consistent Winner     60.032625  0.800895    3129
               Inconsistent Trader -206.074672  0.333333      27
               Mid                   -1.395842  0.235681    3806
fear           Consistent Winner     58.790694  0.521800   33831
               Inconsistent Trader   49.848635  0.320536   16254
               Mid                   46.555958  0.390435   83786
greed          Consistent Winner    169.909842  0.644746    2874
               Inconsistent Trader   55.438298  0.244046    3401
               Mid                   83.719264  0.450423   30014
neutral        Consistent Winner    445.775869  0.809524     294
               Inconsistent Trader   -2.290335  0.320823    3890
               Mid                   12.375271  0.263443    2957

Insights from Consistency-Based Segmentation
* Consistent Winners strongly outperform across all sentiment regimes, maintaining high win rates and positive average PnL. Their performance is particularly strong during greed phases, where both average returns and win rates increase significantly.
* Inconsistent Traders exhibit high variability and underperform during extreme greed, even generating negative average PnL despite optimistic market conditions. This suggests that strong sentiment amplifies poor decision-making for weaker traders.
* During fear regimes, performance gaps between consistent and inconsistent traders narrow, but consistent winners still maintain higher win rates and more stable returns.
* In neutral markets, consistent traders continue to generate strong returns, while mid-tier and inconsistent traders struggle with lower win rates, indicating that skill plays a more critical role when sentiment-driven momentum is absent.

Final Summary — Methodology, Insights & Strategy Recommendations
Methodology

* Cleaned and aligned Hyperliquid trade data with Bitcoin Fear & Greed Index on a daily basis

* Created trader-level metrics including total PnL, win rate, trade frequency, average position size, and PnL volatility

* Segmented traders based on activity (Frequent vs Infrequent) and consistency (Consistent vs Inconsistent)

* Compared performance and risk metrics across different sentiment regimes


Key Insights

* Trader performance is strongly regime-dependent, with Greed environments showing higher average profitability

*  Fear regimes significantly increase volatility and trading activity

* Frequent and consistent traders outperform during extreme sentiment regimes

* Neutral regimes reduce win rates and penalize overtrading

* Sentiment amplifies the performance gap between high-skill and low-skill traders

Strategy Recommendations

* Allocate more capital to frequent and consistent traders during strong Greed regimes

* Apply stricter risk controls during Fear due to elevated volatility

* Reduce trade frequency in Neutral regimes to avoid unnecessary exposure

* Use sentiment regimes as a dynamic filter for capital allocation and risk adjust